# Árboles LLRB

In [1]:
using LLRBTrees

## Árboles

El árbol **LLRB** (*left-leaning red-black* o rojo-negro inclinado a la izquierda) es una **estructura de datos**,
es decir, un objeto que guarda datos y soporta cietas operaciones.

Esta estructura salva datos en **nodos**, que son pares de claves (**keys**) y valores (**values**) y que tienen vínculos a otros nodos. Como cualquier árbol, debe tener un nodo raíz (**root**), que no tiene ningún padre (osea no está vinculado a ningún otro nodo), desde el cuál se puede acceder a todos los otros nodos.

Además, este tipo de árbol es **binario**. Así que cada nodo puede tener dos hijos como máximo. Si no tiene hijos en alguna de sus vacantes, a esta vacante la llamaremos hoja (**leaf**).

El orden de los nodos se hace mediante las claves (keys). La convención es que un hijo izquierdo siempre tiene una clave menor a la de su padre y uno derecho, mayor.

## Árbol básico

Para visualizar esto armemos un árbol básico.

El objeto árbol lo podemos iniciar especificando los tipos de objeto de las claves y valores (las claves deben de tener un orden total):

In [2]:
arbol = LLRBTree{Int, ASCIIString}()

LLRBTrees.LLRBTree{Int64,ASCIIString}


O podemos simplemente indicar los valores de la raíz y julia adivinará los tipos

In [5]:
arbol = LLRBTree(1,"a")

      
   a      


In [6]:
typeof(arbol)

LLRBTrees.LLRBTree{Int64,ASCIIString}

Para añadir un hijo adoptamos el método de Base `push!` y le podemos dar los valores

In [8]:
push!(arbol, 0, "b")

              
       a       
   /           
  b_r             


O un nodo

In [9]:
nodo = TreeNode(2, "c")
push!(arbol, nodo)

              
       a       
    /       \  
   b        c     


## Utilidad

La conveniencia de los árboles binarios de búsqueda (**binary-search trees** o **BST's**) es la velocidad que ofrecen en la búsqueda de una clave.

Para mantener un arreglo de objetos en la memoria se guarda normalmente uno de sus elementos en la memoria y a los demás se les accede por medio de vínculos entre ellos.

Si se tuviera una cadena lineal de objetos (e.d. un unsorted array)

In [10]:
unsorted = LLRBTree(1,1)

nodoactual = unsorted.root
for i in 2:5
    nodoactual.right = TreeNode(i,i)
    nodoactual = nodoactual.right
end

unsorted

                                                                                                                              
                                                               1                                                               
                                                                                               \                               
                                                                                               2_r                             
                                                                                                               \               
                                                                                                               3_r             
                                                                                                                       \       
                                                                                                         

Se necesitarían tantos pasos para llegar al último elemento como elementos tenga el árbol.

In [11]:
@time unsorted[5]

  0.013639 seconds (13.04 k allocations: 531.998 KB)


Nullable(5)

En cambio, en un arbol binario (**BST**) bien ordenado se hace una comparación para ver si la clave buscada es mayor o menor.

Así que en cada comparación se salta uno en promedio la mitad del arbol debajo del nodo.

Por lo que cada búsqueda, inserción o extracción toma aproximadamte el logaritmo base 2 del número de elementos del árbol en pasos.

In [12]:
arbol = LLRBTree(1,1) 
for i in 2:5
    push!(arbol, i,i)
end
arbol

                              
               4               
       /                \      
      2_r               5      
    /      \                   
   1       3                      


In [13]:
@time arbol[5]

  0.000002 seconds (5 allocations: 192 bytes)


Nullable(5)

In [20]:
orderedpairs(arbol)

4-element Array{Any,1}:
 [5,5]  
 [6,6]  
 [10,10]
 [15,15]

Hay que tomar en cuenta que esta comparación en tiempos no es tan justa pues el `getindex()` de los arboles pregunta por el elemento izquierdo primero y luego va al derecho.

## Árboles rojo-negros

Lo que no es tan fácil es mantener el árbol más o menos balanceado.

**Balanceado** quiere decir que cualquier camino de la raíz a una hoja tenga el mismo número de pasos.

Para mantener a los árboles balanceados se han diseñado muchos tipos de árboles auto-balanceados (*self-balancing trees*). Osea que, a medida que se le añaden o quitan elementos se ejecutan otras operaciones que aseguran una especie de balance.


El método que usamos nosotros es una mejora de los árboles rojo-negros (**red-black trees** o **RBT**) que fueron introducidos por **Guibas** y **Robert Sedgewick** hace 30 años.

Los RBT's son una representación binaria de árboles con nodos que pueden tener hasta 4 hijos (**2-3-4 Trees**). En nuestra implementación representamos a un vínculo rojo con el padre como un nodo rojo (al que se la añade "_r" en nuestra representación ASCII) y los nodos vinculados son parte de un gran nodo.

![2-3 vs BST](./23andBSTs.png)
***Fuente: Sedgewick, R. Left-leaning Red-Black Trees. p. 2***

Estos nodos grandes se utilizan para no tener que aumentar o disminuir la altura de solo una rama del árbol cuando se añade o sustrae un nodo.

Por eso, en nuestra implementación, siempre habrá el mismo número de saltos hacia nodos negros, cuando se va de la raíz a cualquier hoja.

En esta implementación se siguen tres principios básicos que hacen que el código sea mas elegante:

* Implementación recursiva
* Solo puede haber 3-nodos y el vínculo solo puede estar a la izquierda (por eso es **left-leaning**)
* Se hacen modificaciones al regresar de la recursión, por ej. se permiten 4-nodos que en el regreso se convierten en tres 2-nodos

## Inserción

Si el nodo final, donde insertamos un hijo, es un 2-nodo (osea tiene max. 2 hijos), lo convertimos en 3-nodo.

In [14]:
arbol = LLRBTree(5,5)
push!(arbol, 10, 10)
push!(arbol, 15, 15)
arbol

              
       10      
    /       \  
   5        15    


In [15]:
push!(arbol,4,4)

                              
               10              
        /               \      
       5                15     
   /                           
  4_r                             


Y si el nodo final es un 3-nodo se puede hacer un rotación a la derecha del nodo padre

In [16]:
push!(arbol,3,3)

                              
               10              
       /                \      
      4_r               15     
    /      \                   
   3       5                      


O a la izquierda

In [17]:
arbol = LLRBTree(5,5)
push!(arbol, 10, 10)
push!(arbol, 15, 15)
arbol

              
       10      
    /       \  
   5        15    


In [18]:
push!(arbol, 6, 6)

                              
               10              
        /               \      
       6                15     
   /                           
  5_r                             
